

# Fast Causal inference

Fast Causal Inference is Tencent's first open-source causal inference project. It is an OLAP-based high-performance causal inference (statistical model) computing library, which solves the performance bottleneck of existing statistical model libraries (R/Python) under big data, and provides causal inference capabilities for massive data execution in seconds and sub-seconds. At the same time, the threshold for using statistical models is lowered through the SQL language, making it easy to use in production environments. At present, it has supported the causal analysis of WeChat-Search, WeChat-Video-Account and other businesses, greatly improving the work efficiency of data scientists.
![Example Image](https://github.com/Tencent/fast-causal-inference/raw/main/docs/images/fast-causal-inference3.png)


# Get started

In [1]:
import time
import pandas as pd

In [2]:
import fast_causal_inference
ais = fast_causal_inference.FCIProvider('all_in_sql')
df = ais.readStarRocks('test_data_small')

In [3]:
df.show()

                                       id            x1            x2           x3           x4           x5 x_long_tail1  x_long_tail2 x_cat1  \
0    0013aaa9-78bd-4421-a485-b06cda2cae7b   -.784221383    2.43174199    .21083256  1.122711457   .627277542   .659296942    .527418381      E   
1    0022fc7e-f3fa-4331-b0f1-878f8203540c    .926981889    -.77896849    .06537978  4.728198792  1.303427799   .746145687    .014150644      C   
2    002c1878-f994-4b11-8d03-62113f621f1e   -.720615247   -.614475543   .379347097  4.051602288  4.502637162   .111069994    .596422673      C   
3    0030d7ee-573d-405e-9609-2c12a364a307   1.342353691    .186825242  1.417049448  2.794842394  4.601687174   .000401808     .72862921      A   
4    003598cf-18c6-4aaf-bd34-73cd33ef347c   -.735976592   1.188873059    .24928749   .095214567  3.735083439   .006746523  19.165992615      D   
..                                    ...           ...           ...          ...          ...          ...          ...   

# know your data

### test data generation

In [4]:
import numpy as np
import pandas as pd
import random
# Set random seed for reproducibility
np.random.seed(0)
n = 10000

################################## generate covariables #############################################
# Generate x1-x5, they come from different distributions and have different variances
x1 = np.random.normal(0, 1, n) # Normal distribution, variance is 1
x2 = np.random.normal(0, 2, n)  # Normal distribution, variance is 4
x3 = np.random.exponential(1, n)  # Exponential distribution, variance is 1
x4 = np.random.exponential(2, n)  # Exponential distribution, variance is 4
x5 = np.random.uniform(0, 5, n)  # Uniform distribution, variance is approximately 1.33
weight = np.random.uniform(0, 1, n) # use for sample reweighing

# Generate x6-x7, they are long-tail distributed data
x_long_tail1 = np.random.pareto(3, n)  # Pareto distribution
x_long_tail2 = np.random.pareto(2, n)  # Pareto distribution

# Generate x_cat1, it is a discrete variable of string type
x_cat1 = np.random.choice(['A', 'B', 'C', 'D', 'E'], n)
###################################################################################################### 



################################## generate exprimental data ######################################### 
# Generate treatment, it is a binary random variable
treatment = np.random.choice([0, 1], n)


# Generate y 
# y is highly correlated with x1, x2, x3,x_long_tail2 and treatment
# and there is heterogeneity on x_cat1 and x1,x2,x4
y_pre = x1 + 2*x2 + 3*x3 + 3*np.log(x_long_tail2+1) + np.random.normal(0, 1, n)
y = y_pre + 5*treatment + treatment*(x1+x2**2+np.log(x4+1)+(x_long_tail1>1))*2 + np.random.normal(0, 2, n)
y[x_cat1 == 'A'] += 3
y[x_cat1 == 'B'] -= 2

# Generate ratio metric: numerator/denominator, for example click/show
numerator_pre = y_pre 
numerator = y 
denominator_pre = x1 + 2*x5 + 3*np.log(x_long_tail1+1) + np.random.normal(0, 1, n)
denominator = denominator_pre + 2*treatment + treatment*(x1)*2 + np.random.normal(0, 2, n)
###################################################################################################### 


################################## generate observational data ####################################### 
# use x1,x2,x3 before
# Generate linear combination for t_ob
linear_combination_t = 1*x1 + 0.2*x2 + 0.5 * x3 + 0.1 * np.random.normal(0, 1, n)

# Convert the linear combination into a probability using the logistic function
prob_t = 1 / (1 + np.exp(-linear_combination_t))

# Generate binary variable t based on the probability
t_ob = np.random.binomial(1, prob_t)

# Generate linear combination for y
linear_combination_y = 0.5 * x1 - 0.25 * x2 + 0.1 * x3 + 0.5 * t_ob + 0.1 * np.random.normal(0, 1, n)

# Generate target variable y
y_ob = linear_combination_y + np.random.normal(0, 1, n)
###################################################################################################### 



# get DataFrame
df = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'x4': x4,
    'x5': x5,
    'x_long_tail1': x_long_tail1,
    'x_long_tail2': x_long_tail2,
    'x_cat1': x_cat1,
    'treatment': treatment,
    't_ob':t_ob,
    'y': y,
    'y_ob': y_ob,
    'numerator_pre':numerator_pre,
    'numerator':numerator,
    'denominator_pre':denominator_pre,
    'denominator':denominator,
    'weight' : weight
})


# show data 
print(df.head())

         x1        x2        x3        x4        x5  x_long_tail1  x_long_tail2 x_cat1  treatment  t_ob          y      y_ob  numerator_pre  \
0  1.764052 -0.404234  0.691733  1.747157  1.567715      0.186325      0.066557      E          1     1  14.341114  0.672431       3.511439   
1  0.400157 -1.666462  4.310034  1.393977  4.755300      0.172270      0.059758      C          0     1  12.022749  2.776448       9.612530   
2  0.978738  3.467200  1.162678  1.188839  1.739792      0.004183      0.002593      E          1     1  44.461318 -1.288468      10.548653   
3  2.240893  0.381298  1.951522  0.235632  0.480281      0.747511      0.114508      E          1     1  17.249192  3.037100       8.715654   
4  1.867558 -0.355621  0.703186  1.544799  3.959299      0.168121      1.927359      D          1     1  20.020350  1.589388       7.982669   

   numerator  denominator_pre  denominator    weight  
0  14.341114         6.554783    11.801924  0.129557  
1  12.022749        10.953917  

Covariables

- `x1`: This variable follows a normal distribution with a mean of 0 and a variance of 1.
- `x2`: This variable also follows a normal distribution, but with a mean of 0 and a larger variance of 4.
- `x3`: This variable is generated from an exponential distribution with a rate parameter of 1, resulting in a variance of 1.
- `x4`: Similar to `x3`, this variable is generated from an exponential distribution, but with a larger rate parameter of 2, resulting in a variance of 4.
- `x5`: This variable is generated from a uniform distribution between 0 and 5, with an approximate variance of 1.33.
- `weight`: This variable is generated from a uniform distribution between 0 and 1 and can be used for sample reweighing.
- `x_long_tail1`: This variable follows a Pareto distribution with a shape parameter of 3, representing a long-tailed distribution.
- `x_long_tail2`: Similarly, this variable follows a Pareto distribution with a shape parameter of 2.
- `x_cat1`: This variable is a discrete variable of string type, randomly chosen from the categories 'A', 'B', 'C', 'D', and 'E'.

Experimental data
- `treatment`: This variable represents the treatment assignment and is a binary random variable.

- `y`: The target variable `y` is highly correlated with `x1`, `x2`, `x3`, `x_long_tail2`, and `treatment`. There is also heterogeneity in the relationship with `x_cat1` and `x1`, `x2`, `x4`. It is generated by adding `y_pre` with treatment effects, interaction terms, and random noise.

- `numerator_pre`: This variable is a precursor to the numerator of a ratio metric and is highly correlated with `y_pre`.

- `numerator`: The numerator of the ratio metric is derived from `numerator_pre` and is highly correlated with `y`.

- `denominator_pre`: This variable is a precursor to the denominator of a ratio metric and is generated based on `x1`, `x5`, and `x_long_tail1`.

- `denominator`: The denominator of the ratio metric is derived from `denominator_pre` and is influenced by treatment effects, interaction terms with `x1`, and random noise.

Observational data 
- `t_ob`: This binary variable is generated based on a linear combination of `x1`, `x2`, and `x3`. The linear combination is converted into a probability using the logistic function, and `t_ob` is generated by sampling from a binomial distribution with the probability.

- `y_ob`: The target variable `y_ob` is generated based on a linear combination of `x1`, `x2`, `x3`, and `t_ob`, along with random noise. It represents the outcome variable in the observational data.


### data description

In [5]:
df = ais.readStarRocks('test_data_small')
df.dtypes

id                 string
x1                  float
x2                  float
x3                  float
x4                  float
x5                  float
x_long_tail1        float
x_long_tail2        float
x_cat1             string
treatment          bigint
t_ob               bigint
y                   float
y_ob                float
numerator_pre       float
numerator           float
denominator_pre    bigint
denominator        bigint
weight              float
day_                 date
dtype: object

In [6]:
df.head(2)

                                     id           x1          x2         x3           x4           x5 x_long_tail1 x_long_tail2 x_cat1 treatment  \
0  0013aaa9-78bd-4421-a485-b06cda2cae7b  -.784221383  2.43174199  .21083256  1.122711457   .627277542   .659296942   .527418381      E         0   
1  0022fc7e-f3fa-4331-b0f1-878f8203540c   .926981889  -.77896849  .06537978  4.728198792  1.303427799   .746145687   .014150644      C         0   

  t_ob             y          y_ob numerator_pre     numerator denominator_pre denominator      weight        day_  
0    1   4.796482619  -1.217059613   6.030571212   4.796482619               1          -1  .816256466  2023-11-03  
1    1  -3.840697601   2.084558089  -2.721380925  -3.840697601               5           5   .11394116  2023-11-03  

In [7]:
# data descirption
df.describe('*')

Ignoring column `id`, whose type `string` is not numeric.
Ignoring column `x_cat1`, whose type `string` is not numeric.
Ignoring column `day_`, whose type `date` is not numeric.


,count,avg,std,min,quantile_0.25,quantile_0.50,quantile_0.75,quantile_0.90,quantile_0.99,max
x1,10000.0,-0.018434,0.987606,-3.740101,-0.689692,-0.026596,0.646518,1.254951,2.310680,3.801660
x2,10000.0,0.021976,1.986209,-8.893264,-1.290912,0.009334,1.352757,2.559644,4.743784,7.196620
x3,10000.0,0.993292,0.987922,0.000439,0.289424,0.693160,1.378255,2.271443,4.440148,9.417911
x4,10000.0,1.985747,1.971388,0.000400,0.564462,1.392178,2.767125,4.609450,9.126206,19.830242
x5,10000.0,2.495959,1.437626,0.000984,1.244424,2.508380,3.736480,4.480969,4.946681,4.999841
x_long_tail1,10000.0,0.515560,0.883486,0.000021,0.099894,0.259858,0.591583,1.191490,3.917824,28.065146
x_long_tail2,10000.0,1.008366,3.048051,0.000002,0.152396,0.410504,1.003290,2.135360,9.445344,156.238517
treatment,10000.0,0.509400,0.499937,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
t_ob,10000.0,0.599100,0.490105,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
y,10000.0,12.359893,12.765055,-13.296274,4.145801,10.012142,17.297520,26.935087,58.579937,148.732617


# AB experiment Analysis

### ttest

- In A/B test analysis, **t-test** is widely used to test whether the average of treatment variant is statistically significantly different from the average of the control variant. However, the mean and variance formula only applied to i.i.d (independent and identically distributed) random variables, and in real business cases our metrics are more complex. Mostly, business metrics are defined as ratios, for example, Clickthrough rate or CTR which is defined as Clicks/Views. Here, we will *utilize the delta method to approximate the variance of the metrics ratio*.
$$t=\frac{\bar X_B-\bar X_A}{\sqrt{Var(\bar X_A)+Var(\bar X_B)}}$$

$$Var(\bar X)=\frac{1}{n}\frac{1}{n-1}\sum_{i=1}^n (X_i-\bar X)^2 $$
- **Delta method** extends the normal approximations of the central limit theorem. Delta method approximates asymptotically normal random variables by applying the Taylor series on the function of random variables. We can estimate the variance of x/y as follows:
$$v=(1/y,-x/y^2)|_{x=\bar x, y=\bar y )}$$

$$\mathop{{M}}\nolimits_{{2 \times 2}}=\frac{1}{n}{ \left[ {\begin{array}{*{20}{c}}
{var(x)}&{cov(x,y)}\\
{cov(x,y)}&{var(y)}\\
\end{array}} \right] }={ \left[ {\mathop{{m}}\nolimits_{{ij}}} \right] }  $$

$$var(x/y)=v*\mathop{{M}}\nolimits_{{2 \times 2}}*v$$

- <font size="4">case1: use deltamethod to do t-test</font>

In [8]:
df.ttest_2samp('avg(numerator)/avg(denominator)','treatment','two-sided')

      mean0     mean1  estimate    stderr t-statistic   p-value     lower     upper
0  0.858954  2.636155  1.777200  0.035934   49.457789  0.000000  1.706763  1.847638

- <font size="4">case2: use deltamethod to do t-test and **CUPED** to reduce variance</font>  


    - We offer a feature in ttest_2samp that utilizes [CUPED (Controlled-experiment Using Pre-Experiment Data)](https://ai.stanford.edu/~ronnyk/2013-02CUPEDImprovingSensitivityOfControlledExperiments.pdf) This technique adjusts metrics using pre-experiment data from both control and treatment groups, aiming to decrease metric variability.
    - To further reduce variance, you can combine multiple metrics using the '+' operator.
    - Additionally, ttest_2samp allows you to perform a t-test grouped by any dimensions of your choice.

In [9]:
df.ttest_2samp('avg(numerator)/avg(denominator)','treatment','two-sided',X='avg(numerator_pre)/avg(denominator_pre)+avg(x1)')

      mean0     mean1  estimate    stderr t-statistic   p-value     lower     upper
0  0.858569  2.636480  1.777910  0.027864   63.806431  0.000000  1.723291  1.832529

In [10]:
df.groupBy('x_cat1').ttest_2samp('avg(numerator)/avg(denominator)','treatment','two-sided',X='avg(numerator_pre)/avg(denominator_pre)')

  x_cat1     mean0     mean1  estimate    stderr t-statistic   p-value     lower     upper
0      D  0.846166  2.606121  1.759955  0.063559   27.690176  0.000000  1.635306  1.884603
1      B  0.440779  2.334651  1.893872  0.058136   32.576650  0.000000  1.779857  2.007887
2      C  0.841622  2.723429  1.881807  0.070967   26.516686  0.000000  1.742629  2.020985
3      E  0.768860  2.584578  1.815718  0.060334   30.094202  0.000000  1.697393  1.934043
4      A  1.339015  2.935081  1.596066  0.065873   24.229521  0.000000  1.466882  1.725250

- <font size="4">case3: ttest_1samp </font>  
    - ttest_1samp can be used in pair test.


In [11]:
df.ttest_1samp('avg(numerator)/avg(denominator)','two-sided')

   estimate    stderr t-statistic   p-value     lower     upper
0  1.901990  0.020915   90.939969  0.000000  1.860993  1.942987

### MWU test

The [Mann–Whitney U test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) is a nonparametric test of the null hypothesis that, for randomly selected values X and Y from two populations, the probability of X being greater than Y is equal to the probability of Y being greater than X.

In [12]:
df.mann_whitney_utest('numerator', 'treatment')

[2379991, 0]

### SRM

**SRM (Sample ratio mismatch)** is an experimental flaw where the expected traffic allocation doesn’t fit with the observed visitor number for each testing variation. We do this using the chi-squared test of independence.

In [13]:
df.srm('numerator', 'treatment', [1,1])

  groupname         f_obs     ratio     chisquare   p-value
0         0  23058.627723  1.000000  48571.698643  0.000000

# Regression-based model

### Linear regression

In statistics, [linear regression](https://en.wikipedia.org/wiki/Linear_regression) is a linear approach for modelling the relationship between a scalar response and one or more explanatory variables (also known as dependent and independent variables). The case of one explanatory variable is called simple linear regression. Written in matrix notation as:
$$y=X\beta+\epsilon$$

#### OLS

In statistics, [ordinary least squares (OLS)](https://en.wikipedia.org/wiki/Ordinary_least_squares) is a type of linear least squares method for choosing the unknown parameters in a linear regression model (with fixed level-one effects of a linear function of a set of explanatory variables) by the principle of least squares: minimizing the sum of the squares of the differences between the observed dependent variable (values of the variable being observed) in the input dataset and the output of the (linear) function of the independent variable.
$$S(\boldsymbol{\beta}) = \sum_{i=1}^n \left| y_i - \sum_{j=1}^p X_{ij}\beta_j\right|^2 = \left\|\mathbf y - \mathbf{X} \boldsymbol \beta \right\|^2.$$
We can leverage **matrix multiplication** to compute Ordinary Least Squares (OLS), which is highly efficient in the OLAP engine we are using.
$$\hat{\boldsymbol{\beta}} = \left( \mathbf{X}^{\operatorname{T}} \mathbf{X} \right)^{-1} \mathbf{X}^{\operatorname{T}} \mathbf y.$$


In [14]:
# ols
df.ols('y~x1+x2+x3+weight', use_bias=True).show()


Call:
  lm( formula = y ~ x1 + x2 + x3 + weight )

Coefficients:
.               Estimate    Std. Error  t value     Pr(>|t|)    
(Intercept)     9.051493    0.257191    35.193599   0.000000    
x1              1.965833    0.116702    16.844859   0.000000    
x2              2.168934    0.058077    37.345693   0.000000    
x3              3.028528    0.116766    25.936792   0.000000    
weight          0.577169    0.400333    1.441721    0.149413    

Residual standard error: 11.524357 on 9995 degrees of freedom
Multiple R-squared: 0.185269, Adjusted R-squared: 0.184943
F-statistic: 568.213021 on 4 and 9995 DF,  p-value: 0.000000



In [15]:
import fast_causal_inference.dataframe.regression as Regression
table = 'test_data_small'
df = fast_causal_inference.readStarRocks(table)
model = Regression.Ols(True)
model.fit('y~x1+x2+t_ob', df)
model.summary()
effect_df = model.effect('x1+x2+x3', df)
effect_df.show()


Call:
  lm( formula = y ~ x1 + x2 + t_ob )

Coefficients:
.               Estimate    Std. Error  t value     Pr(>|t|)    
(Intercept)     11.511121   0.198929    57.865567   0.000000    
x1              1.719414    0.129866    13.239884   0.000000    
x2              2.052233    0.060800    33.753934   0.000000    
t_ob            1.394370    0.265055    5.260684    0.000000    

Residual standard error: 11.890735 on 9996 degrees of freedom
Multiple R-squared: 0.132555, Adjusted R-squared: 0.132295
F-statistic: 509.167914 on 3 and 9996 DF,  p-value: 0.000000

                                       id            x1            x2           x3           x4           x5 x_long_tail1 x_long_tail2 x_cat1  \
0    0000e529-e751-4f13-a04c-642fe2de453e   1.147215774   2.415858286  1.500098323  0.209206196  3.908700241  0.147350970  0.683350882      C   
1    0013aaa9-78bd-4421-a485-b06cda2cae7b  -0.784221383   2.431741990  0.210832560  1.122711457  0.627277542  0.659296942  0.527418381      E 

#### WLS

[Weighted least squares (WLS)](https://en.wikipedia.org/wiki/Weighted_least_squares), also known as weighted linear regression,[1][2] is a generalization of ordinary least squares and linear regression in which knowledge of the unequal variance of observations (heteroscedasticity) is incorporated into the regression. WLS is also a specialization of generalized least squares, when all the off-diagonal entries of the covariance matrix of the errors, are null.
$$  \underset{\boldsymbol\beta}{\operatorname{arg\ min}}\, \sum_{i=1}^{n} w_i \left|y_i - \sum_{j=1}^{m} X_{ij}\beta_j\right|^2 =
  \underset{\boldsymbol\beta}{\operatorname{arg\ min}}\, \left\|W^\frac{1}{2}\left(\mathbf{y} - X\boldsymbol\beta\right)\right\|^2.$$
We can leverage **matrix multiplication** to compute Weighted least squares (WLS), which is highly efficient in the OLAP engine we are using.
$$\hat{\boldsymbol{\beta}} = (X^\textsf{T} W X)^{-1} X^\textsf{T} W \mathbf{y}.$$

In [16]:
import fast_causal_inference.dataframe.regression as Regression
model = Regression.Wls(weight='1', use_bias=True)
model.fit('y~x1+x2+x3', df)
model.summary()
effect_df = model.effect('x1+x2+x3', df)
effect_df.show()


Call:
  lm( formula = y ~ x1 + x2 + x3 )

Coefficients:
.               Estimate    Std. Error  t value     Pr(>|t|)    
(Intercept)     9.337661    0.163560    57.090083   0.000000    
x1              1.966956    0.116706    16.853952   0.000000    
x2              2.167861    0.058076    37.328274   0.000000    
x3              3.031182    0.116757    25.961356   0.000000    

Residual standard error: 11.524979 on 9996 degrees of freedom
Multiple R-squared: 0.185100, Adjusted R-squared: 0.184855
F-statistic: 756.842845 on 3 and 9996 DF,  p-value: 0.000000

                                       id            x1            x2           x3           x4           x5 x_long_tail1 x_long_tail2 x_cat1  \
0    0017e145-28ac-48ba-b6cd-5ee581da8be9   1.437594358  -0.957589743  0.368883827  0.273081111  3.681733397  0.290041805  0.094332574      D   
1    0021469f-676c-4cc5-a75a-27fed536602e  -0.198398897   2.300478296  0.284235675  7.704984329  3.582267588  0.348869983  0.098124044      E   

### logistic model

In statistics, the [logistic model (or logit model)](https://en.wikipedia.org/w/index.php?title=Logistic_regression) is a statistical model that models the probability of an event taking place by having the log-odds for the event be a linear combination of one or more independent variables. In regression analysis, logistic regression (or logit regression) is estimating the parameters of a logistic model (the coefficients in the linear combination). To estimate logistic model, we use **gradient descent**, an optimization algorithm.

$$p(x)=\frac{1}{1+e^{-(\beta_0+\beta_1 x)}}$$

In [17]:
import fast_causal_inference
from fast_causal_inference.dataframe.regression import Logistic
table = 'test_data_small'
df = fast_causal_inference.readStarRocks(table)
X = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_long_tail1', 'x_long_tail2']
Y = 't_ob'
logit = Logistic(tol=1e-6, iter=500)
logit.fit(Y, X, df)
logit.summary()

iter 1 log-likelihood:  -5687.905578239838
iter 2 log-likelihood:  -5627.901418714685
iter 3 log-likelihood:  -5626.94770216139
iter 4 log-likelihood:  -5626.947352745007
The results have converged and the calculation has been stopped


,x,beta
0,intercept,0.083472
1,x1,0.957999
2,x2,0.217600
3,x3,0.534323
4,x4,-0.006258
5,x5,-0.020528
6,x_long_tail1,-0.036267
7,x_long_tail2,0.000232


### IV


[instrumental variables (IV)](https://en.wikipedia.org/wiki/Instrumental_variables_estimation) is a method used in statistics, econometrics, epidemiology, and related disciplines to estimate causal relationships when controlled experiments are not feasible or when a treatment is not successfully delivered to every unit in a randomized experiment. 

The idea behind IV is to use a variable, known as an instrument, that is correlated with the endogenous explanatory variables (the variables that are correlated with the error term), but uncorrelated with the error term itself. This allows us to isolate the variation in the explanatory variable that is purely due to the instrument and thus uncorrelated with the error term, which can then be used to estimate the causal effect of the explanatory variable on the dependent variable.

Here is an example:

1. $t_1 = treatment + X_1 + X_2$
2. $Y = \hat t_1 + X_1 + X_2$

- $X_1$ and $X_2$ are independent variables or predictors.
- $t_1$ is the dependent variable that you are trying to explain or predict.  
- $treatment$ is an independent variable representing some intervention or condition that you believe affects $t_1$. 
- $Y$ is the dependent variable that you are trying to explain or predict
-  $\hat t_1$ is the predicted value of $t_1 from the first equation


We first regress $X_3$ on the treatment and the other exogenous variables $X_1$ and $X_2$ to get the predicted values $\hat t_1$. Then, we replace $t_1$ with $\hat t_1$ in the second equation and estimate the parameters. This gives us the causal effect of $t_1$ on $Y$, purged of the endogeneity problem.

In [18]:
import fast_causal_inference.dataframe.regression as Regression
model = Regression.IV()
model.fit(df,formula='y~(t_ob~treatment)+x1+x2')
model.summary()

df.iv_regression('y~(t_ob~treatment)+x1+x2').show()
df.agg(Regression.iv_regression('y~(t_ob~treatment)+x1+x2')).show()


Call:
  lm( formula = y ~ x1 + x2 + x3 )

Coefficients:
.               Estimate    Std. Error  t value     Pr(>|t|)    
(Intercept)     -2742.218377 501.046885  -5.472978   0.000000    
x1              4578.692890 7548.134776 0.606599    0.544131    
x2              -838.434135 12546.344968 -0.066827   0.946721    
x3              -180.747094 2302.693725 -0.078494   0.937437    

Residual standard error: 9.251235 on 9996 degrees of freedom
Multiple R-squared: 0.474922, Adjusted R-squared: 0.474765
F-statistic: 3013.725638 on 3 and 9996 DF,  p-value: 0.000000


Call:
  lm( formula = y ~ x1 + x2 + x3 )

Coefficients:
.               Estimate    Std. Error  t value     Pr(>|t|)    
(Intercept)     -2742.218377 501.046885  -5.472978   0.000000    
x1              4578.692889 7548.134776 0.606599    0.544131    
x2              -838.434135 12546.344968 -0.066827   0.946721    
x3              -180.747094 2302.693725 -0.078494   0.937437    

Residual standard error: 9.251235 on 9996 degre